In [ ]:
import os
# ⚠ DDE_BACKEND mutlaka deepxde importundan ÖNCE
os.environ["DDE_BACKEND"] = "pytorch"

import numpy as np
import torch
import torch_directml
import deepxde as dde
from scipy.integrate import solve_bvp
import matplotlib.pyplot as plt


# =========================================
# 1) MW-von Kármán nümerik çözücü
# =========================================

def vkd_ode(eta, y):
    """
    MW benzerlik ODE sistemi:
      y[0] = F,  y[1] = F',  y[2] = G,  y[3] = G',  y[4] = H
    """
    f, fp, g, gp, h = y
    return np.vstack([
        fp,
        fp * h + f**2 - g**2,
        gp,
        gp * h + 2 * f * g,
        -2 * f,
    ])


def vkd_bc(ya, yb):
    # f(0)=0, g(0)=1, h(0)=0; f(∞)=0, g(∞)=0  (∞ ~ 10)
    return np.array([ya[0], ya[2] - 1.0, ya[4], yb[0], yb[2]])


def solve_vkd(s_suction=0.0, eta_max=10.0, n_eta=400):
    """
    von Kármán similarity solutions.
    """
    eta_mesh = np.linspace(0.0, eta_max, n_eta)
    y_guess = np.zeros((5, eta_mesh.size))
    y_guess[2, :] = 1.0        # G ~ 1
    y_guess[4, :] = -2.0 * s_suction

    bc = vkd_bc

    sol = solve_bvp(vkd_ode, bc, eta_mesh, y_guess)
    if not sol.success:
        raise RuntimeError(
            f"VKD BVP solver did not converge for smooth disk: "
            + sol.message
        )
    return sol


def generate_obs_profiles_from_vk(Re, vk_sol, r_obs_list=None, L=10.0,
                                  n_z_obs=50):
    """
    Smooth Disk similarity solution generates raw observation data for the PINN.

    Output:
      X_obs : (N,2)  -> [r, z]
      U_obs : (N,1)
      V_obs : (N,1)
      W_obs : (N,1)
      z_obs : (N,1)  (all points' z coordinates)

    r_obs_list:
      - None or a single float: generates data from a single radial point.
      - List/iterable: generates data for all r values in the list
        and concatenates them vertically.
    """
    import numpy as np

    delta = Re ** (-0.5)

    # r list setting: by default [0.1, 1.0]
    if r_obs_list is None:
        r_values = [0.1, 1.0]
    else:
        # If an odd number comes up, convert it to a list
        try:
            iter(r_obs_list)
            if isinstance(r_obs_list, (float, int)):
                r_values = [float(r_obs_list)]
            else:
                r_values = list(r_obs_list)
        except TypeError:
            r_values = [float(r_obs_list)]

    # Equidistant z points in physical space (common for all r's)
    z_line = np.linspace(0.0, L, n_z_obs).reshape(-1, 1)   # (Nz,1)
                                
    y_vk = vk_sol.sol()                         # (5, Nz)
    F = y_vk[0, :]                                         # (Nz,)
    G = y_vk[2, :]
    H = y_vk[4, :]

    X_list, U_list, V_list, W_list, z_list = [], [], [], [], []

    for r_obs in r_values:
        # Physical velocities
        U = r_obs * F
        V = r_obs * (G - 1.0)
        W = H

        # The entrance PINN will see: (r,z)
        X_r = np.hstack([r_obs * np.ones_like(z_line), z_line])  # (Nz,2)

        X_list.append(X_r.astype(np.float32))
        U_list.append(U.reshape(-1, 1).astype(np.float32))
        V_list.append(V.reshape(-1, 1).astype(np.float32))
        W_list.append(W.reshape(-1, 1).astype(np.float32))
        z_list.append(z_line.astype(np.float32))

    X_obs = np.vstack(X_list)
    U_obs = np.vstack(U_list)
    V_obs = np.vstack(V_list)
    W_obs = np.vstack(W_list)
    z_obs = np.vstack(z_list)

    return X_obs, U_obs, V_obs, W_obs, z_obs



def debug_plot_vk_similarity(vk_sol, lambda_slip, eta_slip, outdir):
    eta = vk_sol.x
    y_vk = vk_sol.sol(eta)
    F = y_vk[0, :]
    G = y_vk[2, :]
    H = y_vk[4, :]

    fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharex=True)
    axes[0].plot(eta, F)
    axes[0].set_xlabel(r"$\eta$")
    axes[0].set_ylabel(r"$F(\eta)$")
    axes[0].set_title("Similarity F(η)")

    axes[1].plot(eta, G - 1.0)
    axes[1].set_xlabel(r"$\eta$")
    axes[1].set_ylabel(r"$G(\eta)-1$")
    axes[1].set_title("Similarity G(η)-1")

    axes[2].plot(eta, H)
    axes[2].set_xlabel(r"$\eta$")
    axes[2].set_ylabel(r"$H(\eta)$")
    axes[2].set_title("Similarity H(η)")

    plt.tight_layout()
    fig.savefig(os.path.join(outdir,
                             f"vk_similarity_lambda{lambda_slip:g}_eta{eta_slip:g}.png"),
                dpi=200)
    plt.close(fig)



# =========================================
# 2) PINN
# =========================================

device = torch_directml.device()
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

L = 10.0
r_min = 0.1
r_max = 1.0
geom = dde.geometry.Rectangle([r_min, 0.0], [r_max, L])

lambda_values = [0.0]
eta_values    = [0.0]
s_suction     = 0.0

Re_list = [196]

layer_sizes = [4]
neuron_sizes = 64


# ---------------------------
# Clenshaw–Curtis
# ---------------------------
def clenshaw_curtis_nodes_weights(N, a=-1.0, b=1.0):
    if N < 1:
        raise ValueError("N must be greater than or equal to 1.")
    k = np.arange(0, N + 1)
    x = np.cos(np.pi * k / N)
    w = np.zeros(N + 1, dtype=float)
    if N % 2 == 0:
        m = np.arange(1, N // 2)
        v = np.zeros_like(k, dtype=float)
        for j in m:
            v += (np.cos(2 * j * np.pi * k / N) / (4 * j * j - 1.0))
        w = (2.0 / N) * (1.0 - v - (np.cos(np.pi * k) / (N * N - 1.0)))
    else:
        m = np.arange(1, (N + 1) // 2)
        v = np.zeros_like(k, dtype=float)
        for j in m:
            v += (np.cos(2 * j * np.pi * k / N) / (4 * j * j - 1.0))
        w = (2.0 / N) * (1.0 - v)
    xm = 0.5 * (a + b) + 0.5 * (b - a) * x
    wm = 0.5 * (b - a) * w
    return xm, wm


N_r, N_z = 32, 128
R_nodes, W_r = clenshaw_curtis_nodes_weights(N_r, a=r_min, b=r_max)
Z_nodes, W_z = clenshaw_curtis_nodes_weights(N_z, a=0.0,   b=L)

RR, ZZ = np.meshgrid(R_nodes, Z_nodes, indexing="xy")
Wr, Wz = np.meshgrid(W_r,     W_z,     indexing="xy")
W2D = (Wr * Wz).reshape(-1, 1)

domain_cc = np.column_stack([RR.reshape(-1), ZZ.reshape(-1)]).astype(np.float32)

R_nodes_t = torch.tensor(R_nodes, dtype=torch.float32, device=device).view(1, -1)
Z_nodes_t = torch.tensor(Z_nodes, dtype=torch.float32, device=device).view(1, -1)
W_r_t     = torch.tensor(W_r,     dtype=torch.float32, device=device).view(1, -1)
W_z_t     = torch.tensor(W_z,     dtype=torch.float32, device=device).view(1, -1)


def cc_weight_rz(r, z):
    dist_r = torch.abs(r - R_nodes_t)     # (B, Nr+1)
    idx_r  = torch.argmin(dist_r, dim=1, keepdim=True)
    dist_z = torch.abs(z - Z_nodes_t)
    idx_z  = torch.argmin(dist_z, dim=1, keepdim=True)
    w_r = torch.gather(W_r_t.repeat(r.shape[0], 1), 1, idx_r)
    w_z = torch.gather(W_z_t.repeat(z.shape[0], 1), 1, idx_z)
    return w_r * w_z  # (B,1)


# ---------------------------
# PDE + slip & far-field residual
# ---------------------------
def make_pde(Re, r_key=1.0):
    def pde(x, y):
        r = x[:, 0:1]
        z = x[:, 1:2]

        U = y[:, 0:1]
        V = y[:, 1:2]
        W = y[:, 2:3]
        P = y[:, 3:4]

        U_r = dde.grad.jacobian(y, x, i=0, j=0)
        U_z = dde.grad.jacobian(y, x, i=0, j=1)
        V_r = dde.grad.jacobian(y, x, i=1, j=0)
        V_z = dde.grad.jacobian(y, x, i=1, j=1)
        W_r = dde.grad.jacobian(y, x, i=2, j=0)
        W_z = dde.grad.jacobian(y, x, i=2, j=1)
        P_r = dde.grad.jacobian(y, x, i=3, j=0)
        P_z = dde.grad.jacobian(y, x, i=3, j=1)

        U_rr = dde.grad.jacobian(U_r, x, j=0)
        U_zz = dde.grad.jacobian(U_z, x, j=1)
        V_rr = dde.grad.jacobian(V_r, x, j=0)
        V_zz = dde.grad.jacobian(V_z, x, j=1)
        W_rr = dde.grad.jacobian(W_r, x, j=0)
        W_zz = dde.grad.jacobian(W_z, x, j=1)

        # Rotating-disk PDEs
        eq1 = (1.0 / r) * (U + r * U_r) + W_z
        eq2 = U * U_r + W * U_z - (V + r) ** 2 / r - (1.0 / Re) * (U_zz + (1.0 / r) * U_r + U_rr - U / (r ** 2)) + P_r
        #eq2 = U * U_r + W * U_z - V**2 / r - (1.0 / Re) * (U_zz + (1.0 / r) * U_r + U_rr - U / (r ** 2))
        eq3 = U * V_r + W * V_z + (U * V) / r + 2.0 * U - (1.0 / Re) * (V_zz + (1.0 / r) * V_r + V_rr - V / (r ** 2))
        #eq3 = U * V_r + W * V_z + (U * V) / r  - (1.0 / Re) * (V_zz + (1.0 / r) * V_r + V_rr - V / (r ** 2))
        eq4 = U * W_r + W * W_z + P_z - (1.0 / Re) * ((1.0 / r) * W_r + W_rr + W_zz)

        # Clenshaw–Curtis weight (for PDE part)
        w_cc = cc_weight_rz(r, z)
        s = torch.sqrt(torch.clamp(w_cc, min=1e-30))

        # --- Slip BC (z=0): U = 0, V = 0, W=0 ---
        mask_bottom = (torch.abs(z) < 1e-20).float()
        eq5 = mask_bottom * (U - 0)
        eq6 = mask_bottom * (V - 0)
        eq7 = mask_bottom * W

        # --- Far field (z=L): U=0, V=-r, W=0 ---
        mask_top = (torch.abs(z - L) < 1e-20).float()
        eq8  = mask_top * U
        eq9  = mask_top * (V + r)
        # eq10 = mask_top * W

        residuals = [
            s * eq1, s * eq2, s * eq3, s * eq4,
            s * eq5, s * eq6, s * eq7, s * eq8, s * eq9,
        ]

        return residuals

    return pde

# =========================================
# 3) Training cycle
# =========================================
base_dir = "Outputs_PINN_sweep_dif-activations"
os.makedirs(base_dir, exist_ok=True)

print(f"\n===BVP: being solved ===")
vk_sol = solve_vkd(
                    s_suction=s_suction,
                    eta_max=10.0,
                    n_eta=400)

slip_dir = os.path.join(
    base_dir, f"VK_ref_Re196_64x4_tanh"
)
os.makedirs(slip_dir, exist_ok=True)

# Similarity profillerini (η üzerinde) kaydetmek istersen:
debug_plot_vk_similarity(vk_sol, slip_dir)
activation_funcs= ["tanh"]

for Re in Re_list:
    for l in layer_sizes:
        for act in activation_funcs:

            vk_sol = solve_vkd(
                s_suction=0.0,
                eta_max=10.0,
                n_eta=400,
            )

            # --- MW çözümünden kritik parametreler ---
            y0 = vk_sol.sol(0.0)
            f0, fp0, g0, gp0, h0 = y0

            # r = 1 için:
            # u(r=1,z) = F(eta)  -> u_z(0) = F'(0) = fp0
            # v(r=1,z) = G(eta)-1 -> v_z(0) = G'(0) = gp0
            u_z0_ref = float(fp0)
            v_z0_ref = float(gp0)

            # Uzak alan: z = L için eta = min(L, eta_max)
            eta_L = min(L, vk_sol.x[-1])
            yL = vk_sol.sol(eta_L)
            hL = yL[4]
            w_L_ref = float(hL)


            pde = make_pde(
                            Re,
                            u_z0_ref=u_z0_ref,
                            v_z0_ref=v_z0_ref,
                            w_L_ref=w_L_ref,
                            r_key=1.0,   # r=1 doğrusu
                        )

            # ---- MW çözümünden HAM obs verisi (sadece z üzerinden) ----
            X_obs, U_obs, V_obs, W_obs, z_obs = generate_obs_profiles_from_vk(
                Re=Re,
                vk_sol=vk_sol,
                r_obs_list=[1.0],
                L=L,
                n_z_obs=100,
            )

            print(f"[OBS DEBUG] Re={Re}")
            print("  X_obs shape:", X_obs.shape)
            print("  z range     :", float(z_obs.min()), "→", float(z_obs.max()))
            print("  U min/max   :", float(U_obs.min()), "→", float(U_obs.max()))
            print("  V min/max   :", float(V_obs.min()), "→", float(V_obs.max()))
            print("  W min/max   :", float(W_obs.min()), "→", float(W_obs.max()))

            # z üzerindeki U,V,W profilleri (kontrol için)
            fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharex=True)
            axes[0].plot(z_obs, U_obs[:, 0])
            axes[0].set_xlabel("z")
            axes[0].set_ylabel("U(r=1,z)")
            axes[0].set_title("Numeric U profiles")

            axes[1].plot(z_obs, V_obs[:, 0])
            axes[1].set_xlabel("z")
            axes[1].set_ylabel("V(r=1,z)")
            axes[1].set_title("Numeric V profiles")

            axes[2].plot(z_obs, W_obs[:, 0])
            axes[2].set_xlabel("z")
            axes[2].set_ylabel("W(r=1,z)")
            axes[2].set_title("Numeric W profiles")

            plt.tight_layout()
            fig.savefig(os.path.join(
                slip_dir,
                f"vk_obs_profiles_z_Re{int(Re)}.png"
            ), dpi=200)
            plt.close(fig)

            # PointSetBC: PINN'e verilen tek şey bu ham (r,z) + U,V,W datası
            bc_U = dde.icbc.PointSetBC(X_obs, U_obs, component=0)
            bc_V = dde.icbc.PointSetBC(X_obs, V_obs, component=1)
            bc_W = dde.icbc.PointSetBC(X_obs, W_obs, component=2)
            bcs = [bc_U, bc_V, bc_W]

            # Data (anchors=CC + obs BC)
            data = dde.data.PDE(
                geom, pde, bcs,
                num_domain=0,
                num_boundary=0,
                num_test=2000,
                anchors=domain_cc,
            )

            delta = Re ** (-0.5)

            # --- Sadece PINN için feature/output transform ---
            def feature_transform(x):
                r = x[:, 0:1]
                z = x[:, 1:2]
                eta = z / delta    # sadece ağın girdisi
                return torch.cat([r, eta], dim=1)

            layers = [2] + [64] * l + [4]
            net = dde.nn.FNN(layers, act, "Glorot uniform")
            net.apply_feature_transform(feature_transform)

            def output_transform(x, y):
                r = x[:, 0:1]
                U = r*y[:, 0:1]
                V = r*y[:, 1:2]
                W = delta * y[:, 2:3]
                P = (delta ** 2) * y[:, 3:4]
                return torch.cat([U, V, W, P], dim=1)

            net.apply_output_transform(output_transform)
            net.to(device)

            output_dir = os.path.join(
                base_dir,
                f"MW_Re{int(Re)}_64x{l}_{act}"
            )
            os.makedirs(output_dir, exist_ok=True)
            lambda_data = 0.5
            key_weight = 10.0
            loss_weights = [1.0] * 9  + [lambda_data] * 3

            model = dde.Model(data, net)
            model.compile("adam", lr=5e-4, loss_weights=loss_weights)

            losshistory_adam, train_state_adam = model.train(
                iterations=10000,
                model_save_path=os.path.join(output_dir, "model_adam"),
                verbose=1,
                display_every=500,
            )
            dde.saveplot(
                losshistory_adam, train_state_adam,
                issave=True, isplot=True, output_dir=output_dir
            )

In [ ]:
import os
os.environ["DDE_BACKEND"] = "pytorch"

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

import torch
try:
    import torch_directml
    device = torch_directml.device()
except Exception:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import deepxde as dde

torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

# =========================
# 1) von Kármán numerical solution
# =========================

L  = 10.0    # z_max
Nz = 600

def vkd_ode(eta, y):
    f, fp, g, gp, h = y
    return np.vstack([
        fp,
        fp * h + f**2 - g**2,
        gp,
        gp * h + 2.0 * f * g,
        -2.0 * f,
    ])

def vkd_bc(ya, yb):
    return np.array([
        ya[0],       # f(0)=0
        ya[2] - 1.0, # g(0)=1
        ya[4],       # h(0)=0
        yb[0],       # f(∞)=0
        yb[2],       # g(∞)=0
    ])

def solve_vkd_and_get_UVW(z_max=L, Nz=Nz):
    eta_init = np.linspace(0.0, 10.0, 100)
    y_guess  = np.zeros((5, eta_init.size))
    y_guess[2, :] = 1.0

    sol = solve_bvp(vkd_ode, vkd_bc, eta_init, y_guess)
    if not sol.success:
        raise RuntimeError("BVP solution did not converge: " + sol.message)

    z = np.linspace(0.0, z_max, Nz)
    Y = sol.sol(z)
    U = Y[0]          # U
    V = Y[2] - 1.0    # V = g-1
    W = Y[4]          # W
    return z, U, V, W

z_num, U_num, V_num, W_num = solve_vkd_and_get_UVW()

# =========================
# 2) PINN: Load all networks from the Re list and add scatter
# =========================

Re_list = [196, 259, 391, 471.0, 555.0, 568.0, 615.0, 648.0, 713.0, 752.0]
r_min, r_max = 0.1, 1.0
L = 10.0

base_dir = "Outputs_PINN_Different_Re"
compare_fig_dir = "./VKD_PINN_compare_raw"
os.makedirs(compare_fig_dir, exist_ok=True)

# --- Weight loading functions ---
def try_restore_ckpt(model, outdir):
    for fname in sorted(os.listdir(outdir)):
        if fname.endswith(".ckpt"):
            ckpt_path = os.path.join(outdir, fname)
            try:
                model.restore(ckpt_path, verbose=0)
                return True, ("ckpt", ckpt_path)
            except Exception:
                pass
    return False, None

def try_load_pt(net, outdir):
    for name in [
        "model_adam-10000.pt",
        "model_adam-1500.pt",
        "model_adam-5000.pt",
        "model_adam.pt",
        "model_final.pt",
        "model_bfgs-3810.pt",
    ]:
        p = os.path.join(outdir, name)
        if os.path.isfile(p):
            state = torch.load(p, map_location=device)
            for cand in [
                state,
                isinstance(state, dict) and state.get("model_state_dict"),
                isinstance(state, dict) and state.get("state_dict"),
                isinstance(state, dict) and state.get("net"),
            ]:
                if isinstance(cand, dict):
                    try:
                        net.load_state_dict(cand, strict=True)
                        return True, ("pt", p)
                    except Exception:
                        pass
    return False, None

# --- Create three figures: U, V, W ---
figU, axU = plt.subplots(figsize=(6.8, 5.2), dpi=130)
figV, axV = plt.subplots(figsize=(6.8, 5.2), dpi=130)
figW, axW = plt.subplots(figsize=(6.8, 5.2), dpi=130)

# Numeric curves
axU.plot(z_num, U_num, lw=2.0, label="von Kármán numeric")
axV.plot(z_num, V_num, lw=2.0, label="von Kármán numeric")
axW.plot(z_num, W_num, lw=2.0, label="von Kármán numeric")

markers = ["o", "s", "^", "D", "P", "X", "v", "<", ">", "*"]
# --- For each Re, get r=1 profile from PINN, select 50 points, add as scatter ---

for i, Re in enumerate(Re_list):
    Re_val = float(Re)
    delta = Re_val ** (-0.5) 

    print(f"\n=== Re = {Re_val:.1f} ===")
    print(f"delta = {delta:.6f}")

    m = markers[i % len(markers)]  # different marker for each Re

    # Network architecture
    layers = [2] + [64] * 4 + [4]
    net = dde.nn.FNN(layers, "tanh", "Glorot uniform")

    # Feature and output transforms
    def feature_transform(x, delta=delta):
        r = x[:, 0:1]
        z = x[:, 1:2]
        eta = z / delta
        return torch.cat([r, eta], dim=1)

    def output_transform(x, y, delta=delta):
        r = x[:, 0:1]
        U = r * y[:, 0:1]           # U = r f(η)
        V = r * y[:, 1:2]           # V = r (g(η)-1)
        W = delta * y[:, 2:3]       # W = δ w̄(η)
        P = (delta**2) * y[:, 3:4]
        return torch.cat([U, V, W, P], dim=1)

    net.apply_feature_transform(feature_transform)
    net.apply_output_transform(output_transform)
    net.to(device)

    # DeepXDE
    geom = dde.geometry.Rectangle([r_min, 0.0], [r_max, L])
    dummy_data = dde.data.PDE(geom, lambda x, y: [y], [], num_domain=0, num_boundary=0)
    model = dde.Model(dummy_data, net)
    model.compile("adam", lr=1e-3)

    # Load weights
    output_dir = os.path.join(base_dir, f"output_vK_Re{int(Re_val)}")
    if not os.path.isdir(output_dir):
        raise FileNotFoundError(f"Directory not found: {output_dir}")

    ok, src = try_restore_ckpt(model, output_dir)
    if not ok:
        ok, src = try_load_pt(net, output_dir)
    if not ok:
        raise FileNotFoundError(
            f"Weights not found: No .ckpt or model_adam-*.pt under {output_dir}"
        )

    print(f"Loaded: {src[0]} -> {src[1]}")

    # PINN prediction along r=1, z ∈ [0,L]
    z_eval = np.linspace(0.0, L, Nz).astype(np.float32)
    r_eval = np.ones_like(z_eval, dtype=np.float32) * 1.0  # r=1
    X_eval = np.stack([r_eval, z_eval], axis=1)

    Y_pred = model.predict(X_eval)  # (Nz, 4) -> [U, V, W, P]
    U_pinn = Y_pred[:, 0]
    V_pinn = Y_pred[:, 1]
    W_pinn = Y_pred[:, 2] 

    # Select 50 sample points (equally spaced indices along z axis)
    idx = np.linspace(0, Nz - 1, 50, dtype=int)

    label = f"PINN Re={int(Re_val)}"
    axU.scatter(z_eval[idx], U_pinn[idx], s=20, alpha=0.8, marker=m, label=label)
    axV.scatter(z_eval[idx], V_pinn[idx], s=20, alpha=0.8, marker=m, label=label)
    axW.scatter(z_eval[idx], W_pinn[idx], s=20, alpha=0.8, marker=m, label=label)


# --- Final adjustments and saving ---
for ax, fig, name, ylabel in [
    (axU, figU, "U", r"$\hat{u}(r=1,z)$"),
    (axV, figV, "V", r"$\hat{v}(r=1,z)$"),
    (axW, figW, "W", r"$\hat{w}(r=1,z)$"),
]:
    ax.set_xlabel("z")
    ax.set_ylabel(ylabel)
    ax.grid(True, linestyle="--", color="black", alpha=0.35)
    ax.legend(fontsize=7, ncol=2)
    fig.tight_layout()
    fig.savefig(os.path.join(compare_fig_dir, f"vK_vs_PINN_{name}.png"), dpi=130)
    fig.savefig(os.path.join(compare_fig_dir, f"vK_vs_PINN_{name}.pdf"), dpi=130)
    plt.close(fig)

print(f"\nComparison figures saved to: {compare_fig_dir}")
